## Advanced missing values imputation techniques
In this notebook, we explore some missing value imputation techniques for tabular data on different public datasets. 
The goal of this workshop is to introduce these different techniques and their implementations in open source libraries.


For each dataset, we compare different missing value imputation techniques with a naive baseline in terms of machine learning efficiency: we train a machine learning model that performs a downstream prediction task and we observe the change in performance between the naive baseline and the imputed versions. 

The main library used in this workshop is **Hyperimpute**, it groups various imputation techniques from naive approaches (e.g. mean, most frequent value) to approcahes based on generative models.


### Install and load necessary libraries

In [ ]:
!pip install category-encoders
!pip install hyperimpute
!pip install pypots
!pip install autogluon

In [ ]:
import category_encoders as ce
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### IMPUTATION ###
from hyperimpute.plugins.imputers import Imputers
from pypots.utils.metrics import calc_mae, calc_rmse

# ML efficiency
from autogluon.tabular import TabularPredictor

TODO: Description of types of missing

### Mixed-type Tabular + Missing Not at Random
In this section, we explore imputation for a tabular dataset with mixed-type features (numerical and categorical) that contains missing data not at random. Specifically, in this case, we are looking at the Cirrhosis Patient Survival Prediction dataset. The downstream task we use for machine learning efficiency evaluation is the survival status of the patient.

In [ ]:
# Load MNAR data
cirrhosis = pd.read_csv("data/cirrhosis.csv")
categorical_columns = [
    "Status",
    "Drug",
    "Sex",
    "Ascites",
    "Hepatomegaly",
    "Spiders",
    "Edema",
    "Stage",
]

Let's have a first glance of the data, more specifically at the amount of missing values:

In [ ]:
cirrhosis.isnull().sum()

We would like to check if the data has the issue of class imbalance to decide which evaluation metric is more appropriate for our analysis.

In [ ]:
cirrhosis.Status.value_counts()

Also let's check the class imbalance for rows with any missing values.

In [ ]:
cirrhosis[cirrhosis.isnull().any(axis=1)].Status.value_counts()

And the ratio.

In [ ]:
cirrhosis[
    cirrhosis.isnull().any(axis=1)
].Status.value_counts() / cirrhosis.Status.value_counts()

#### Ordinal encoding for categorical features

In [ ]:
encoder = ce.OrdinalEncoder(handle_missing="return_nan")
encoded_df = encoder.fit_transform(cirrhosis)

In [ ]:
encoded_df

#### Train/test split

In [ ]:
train_df, test_df = train_test_split(
    encoded_df, test_size=0.2, stratify=cirrhosis[["Status", "Sex"]], random_state=42
)

#### Compute machine learning efficiency
We use machine learning efficiency to evaluate the performance of the impytation technique. The idea is to compare the performance of a machine learning model on the baseline (without data imputation) against the imputed versions.

In [ ]:
def cal_ml_efficiency(train, test, label, eval_metric="f1_macro"):
    """
    Evaluate machine learning efficiency using AutoGluon's TabularPredictor.

    Parameters:
    - train:        pd.DataFrame, the training dataset.
    - test:         pd.DataFrame, the testing dataset.
    - label:        str, the label column.
    - eval_metric:  str, the evaluation metric to use.

    Returns:
    - dict, evaluation results.
    """
    predictor = TabularPredictor(label=label, eval_metric=eval_metric)

    # Define the hyperparameters to use only NN_Torch
    hyperparameters = {
        "NN_TORCH": {
            "num_layers": 3,  # Number of layers in the model
            "hidden_size": 128,  # Size of the hidden layers
        },
    }

    # Fit the model with only NN_Torch
    predictor.fit(train, hyperparameters=hyperparameters, presets="best_quality")

    return predictor.evaluate(test)

#### Baseline
The ML efficiency evaluation on the baseline case.

In [ ]:
baseline_res = cal_ml_efficiency(
    train=train_df, test=test_df, label="Status", eval_metric="f1_macro"
)

#### Impute missing values

In [ ]:
def impute_missing_values(
    algo,
    df_missing,
    train_indices,
    test_indices,
    random_state=42,
    categorical_cols=None,
):
    """
    Impute missing values in a DataFrame using the specified algorithm.

    Parameters:
    - algo:             str, the imputation algorithm to use.
    - df_missing:       pd.DataFrame, the original DataFrame with missing values.
    - train_indices:    list, the indices of the training set.
    - test_indices:     list, the indices of the testing set.
    - random_state:     int, the random seed to use.
    - categorical_cols: list, the names of the categorical columns.

    Returns:
    - pd.DataFrame, the imputed train and test DataFrames.
    """
    print("Available imputing algorithms:\n" + "\n".join(Imputers().list()))

    imputer = Imputers().get(algo, random_state=random_state)
    df_imp = imputer.fit_transform(df_missing.copy())

    # assign the column names back
    df_imp.columns = df_missing.columns

    if categorical_cols:
        df_imp[categorical_cols] = df_imp[categorical_cols].round()

    # Split the data into training and testing sets
    train_imp = df_imp.loc[train_indices]
    test_imp = df_imp.loc[test_indices]

    return train_imp, test_imp

#### Miss Forest
Missing values imputation using Miss Forest: It uses a random forest trained on the observed values of a data matrix to predict the missing values. It can be used to impute continuous and/or categorical data including complex interactions and non-linear relations.


In [ ]:
train_imputed_mf, test_imputed_mf = impute_missing_values(
    algo="missforest",
    df_missing=encoded_df,
    train_indices=train_df.index,
    test_indices=test_df.index,
    categorical_cols=categorical_columns,
)

In [ ]:
mf_res = cal_ml_efficiency(
    train=train_imputed_mf, test=test_imputed_mf, label="Status", eval_metric="f1_macro"
)

#### GAIN
GAIN is a missing values imputation method that uses GANs.  The generator observes some components of a real data vector, imputes the missing components conditioned on what is actually observed, and outputs a completed vector. The discriminator then takes a completed vector and attempts to determine which components were actually observed and which were imputed.

In [ ]:
train_imputed_gain, test_imputed_gain = impute_missing_values(
    algo="gain",
    df_missing=encoded_df,
    train_indices=train_df.index,
    test_indices=test_df.index,
    categorical_cols=categorical_columns,
)

In [ ]:
gain_res = cal_ml_efficiency(
    train=train_imputed_gain,
    test=test_imputed_gain,
    label="Status",
    eval_metric="f1_macro",
)

#### MIRACLE
MIRACLE is causally-aware imputation algorithm. It iteratively refines the imputation of a baseline by simultaneously modeling the missingness generating mechanism, encouraging imputation to be consistent with the causal structure of the data.

In [ ]:
train_imputed_miracle, test_imputed_miracle = impute_missing_values(
    algo="miracle",
    df_missing=encoded_df,
    train_indices=train_df.index,
    test_indices=test_df.index,
    categorical_cols=categorical_columns,
)

In [ ]:
miracle_res = cal_ml_efficiency(
    train=train_imputed_miracle,
    test=test_imputed_miracle,
    label="Status",
    eval_metric="f1_macro",
)

#### MICE
MICE is an algorithm that performs multiple imputations based on regularized linear regression.

In [ ]:
train_imputed_mice, test_imputed_mice = impute_missing_values(
    algo="mice",
    df_missing=encoded_df,
    train_indices=train_df.index,
    test_indices=test_df.index,
    categorical_cols=categorical_columns,
)

In [ ]:
mice_res = cal_ml_efficiency(
    train=train_imputed_mice,
    test=test_imputed_mice,
    label="Status",
    eval_metric="f1_macro",
)

#### Comparison

In [ ]:
import matplotlib.pyplot as plt

def plot(result, title, y_limit=None, log_scale=False):
	"""
	Plots a bar chart for comparing evaluation metrics across different methods.

	Parameters:
	"""
    # Create a color palette
    colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854']

    # Plotting with a color palette and edges
    ax = result.plot(kind="bar", figsize=(12, 7), width=0.8, color=colors[:result.shape[1]], edgecolor='black')

    # Adding titles and labels with bigger fonts
    plt.title(title, fontsize=16, weight='bold')
    plt.xlabel("Metrics", fontsize=12)
    plt.ylabel("Scores", fontsize=12)

    # Rotating x-axis labels for better readability
    plt.xticks(rotation=45, ha='right', fontsize=11)

    # Set y-axis limits
    if y_limit:
        plt.ylim(y_limit)

    # Set y-axis to logarithmic scale if log_scale is True
    if log_scale:
        plt.yscale('log')

    # Add grid lines
    plt.grid(True, which='major', axis='y', linestyle='--', alpha=0.7)

    # Add a legend with bigger fonts
    plt.legend(title="Methods", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=11, title_fontsize=12)

    # Adjust layout to ensure the plot fits well
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
res = pd.DataFrame({
    "Baseline": baseline_res,
    "Miss Forest": mf_res,
    "GAIN": gain_res,
    "MIRACLE": miracle_res,
    "MICE": mice_res,
}).set_index("Metric")

In [ ]:
plot(result=res, title='Classification Metrics', y_limit=(0.8,1), log_scale=False)

### TO DO: COMMENT ON RESULTS

### Numerical Tabular + Missing Completely at Random
In this section we explore missing value imputation for a numerical dataset with randomly inserted missing values. For this we use the Wisconson breast cancer dataset.

https://www.kaggle.com/datasets/saurabhbadole/breast-cancer-wisconsin-state

In [ ]:
df_ori = (
    pd.read_csv(
        "data/breast-cancer-wisconsin.data",
        delimiter=",",
        header=None,
        names=[
            "Sample code number",
            "Clump Thickness",
            "Uniformity of Cell Size",
            " Uniformity of Cell Shape",
            "Marginal Adhesion",
            "Single Epithelial Cell Size",
            "Bare Nuclei",
            "Bland Chromatin",
            "Normal Nucleoli",
            "Mitoses",
            "Class",
        ],
    )
    .drop(columns=["Sample code number"])
    .replace("?", np.nan)
    .apply(pd.to_numeric, errors="coerce")
)

Let's take a quick look at the dataset

In [ ]:
df_ori.Class.value_counts()

Now let's generate some artificial missing values.

#### Generate missing values at random

In [ ]:
def gene_missing_at_random(data, missing_rate, label_col, random_state=42):
    """
    Generate missing data at random in a DataFrame.

    Parameters:
    - data:         pd.DataFrame, the input DataFrame.
    - missing_rate: float, the proportion of data to be set as missing (0 <= missing_rate <= 1).
    - label_col:    str, the name of the label column, which should not be set as missing.
    - random_state: int, the random seed to use.

    Returns:
    - pd.DataFrame, the original DataFrame.
    - pd.DataFrame, with missing values introduced.
    """
    np.random.seed(random_state)

    # Separate the feature columns and the label column
    feature_columns = data.drop(columns=[label_col])
    label_column = data[label_col]

    # Masking the feature columns with missing values
    mask = np.random.rand(*feature_columns.shape) < missing_rate
    features_with_missing = feature_columns.mask(mask)

    return data, pd.concat([features_with_missing, label_column], axis=1)

In [ ]:
df_ori, df_miss = gene_missing_at_random(df_ori, 0.3, "Class")

In [ ]:
train_mar, test_mar = train_test_split(
    df_miss, test_size=0.3, stratify=df_miss[["Class"]], random_state=42
)

#### Baseline

In [ ]:
mcar_base_res = cal_ml_efficiency(
    train=train_mar,
    test=test_mar,
    label="Class",
    eval_metric="f1_macro",
)

#### Miss Forest

In [ ]:
train_imputed_mf, test_imputed_mf = impute_missing_values(
    algo="missforest",
    df_missing=df_miss,
    train_indices=train_mar.index,
    test_indices=test_mar.index,
)

In [ ]:
mf_res = cal_ml_efficiency(
    train=train_imputed_mf,  # .clip(lower=1, upper=10).round()
    test=test_imputed_mf,  # .clip(lower=1, upper=10).round()
    label="Class",
    eval_metric="f1_macro",
)

#### GAIN

In [ ]:
train_imputed_gain, test_imputed_gain = impute_missing_values(
    algo="gain",
    df_missing=df_miss,
    train_indices=train_mar.index,
    test_indices=test_mar.index,
)

In [ ]:
gain_res = cal_ml_efficiency(
    train=train_imputed_gain,  # .clip(lower=1, upper=10).round()
    test=test_imputed_gain,  # .clip(lower=1, upper=10).round()
    label="Class",
    eval_metric="f1_macro",
)

#### MIRACLE

In [ ]:
train_imputed_miracle, test_imputed_miracle = impute_missing_values(
    algo="miracle",
    df_missing=df_miss,
    train_indices=train_mar.index,
    test_indices=test_mar.index,
)

In [ ]:
miracle_res = cal_ml_efficiency(
    train=train_imputed_miracle,  # .clip(lower=1, upper=10).round()
    test=test_imputed_miracle,  # .clip(lower=1, upper=10).round()
    label="Class",
    eval_metric="f1_macro",
)

#### MICE

In [ ]:
train_imputed_mice, test_imputed_mice = impute_missing_values(
    algo="mice",
    df_missing=df_miss,
    train_indices=train_mar.index,
    test_indices=test_mar.index,
)

In [ ]:
mice_res = cal_ml_efficiency(
    train=train_imputed_mice,  # .clip(lower=1, upper=10).round()
    test=test_imputed_mice,  # .clip(lower=1, upper=10).round()
    label="Class",
    eval_metric="f1_macro",
)

#### Comparison

In [ ]:
res = {
    "Baseline": mar_base_res,
    "Miss Forest": mf_res,
    "GAIN": gain_res,
    "MIRACLE": miracle_res,
    "MICE": mice_res,
}

In [ ]:
pd.DataFrame(res)

### Compute metrics
TO DO: Explain what these metrics mean.

In [ ]:
def cal_metrics(original, imputed, metric="all"):
    """
    Calculate imputation quality metrics for given original and imputed datasets.

    Parameters:
    - original: pd.DataFrame, the original dataset with missing values.
    - imputed:  pd.DataFrame, the imputed dataset.
    - metric:   str, the metric to calculate ('mae' or 'rmse').

    Returns:
    - float, the calculated metric value.
    """
    valid_rows = ~original.isnull().any(axis=1)
    ori_np = original[valid_rows].to_numpy()
    imp_np = imputed[valid_rows].to_numpy()

    metrics = {
        "mae": calc_mae,
        "rmse": calc_rmse,
    }

    if metric == "all":
        return {name: func(ori_np, imp_np) for name, func in metrics.items()}
    elif metric in metrics:
        return metrics[metric](ori_np, imp_np)
    else:
        raise ValueError(f"Invalid metric '{metric}'. Choose 'mae', 'rmse', or 'all'.")

### TO DO: PUT THESE RESULTS TOGETHER IN A BARPLOT

In [ ]:
cal_metrics(
    original=df_ori,
    imputed=pd.concat([train_imputed_mf, test_imputed_mf])
    .sort_index()
    .clip(lower=1, upper=10)
    .round(),
)

In [ ]:
cal_metrics(
    original=df_ori,
    imputed=pd.concat([train_imputed_gain, test_imputed_gain])
    .sort_index()
    .clip(lower=1, upper=10)
    .round(),
)

In [ ]:
cal_metrics(
    original=df_ori,
    imputed=pd.concat([train_imputed_miracle, test_imputed_miracle])
    .sort_index()
    .clip(lower=1, upper=10)
    .round(),
)

In [ ]:
cal_metrics(
    original=df_ori,
    imputed=pd.concat([train_imputed_mice, test_imputed_mice])
    .sort_index()
    .clip(lower=1, upper=10)
    .round(),
)

In [ ]:
cal_metrics(
	original=df_ori,
	imputed=pd.concat([train_imputed_miracle, test_imputed_miracle]).sort_index().clip(lower=1, upper=10).round(),
)